In [1]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import datetime
from pathlib import Path

In [2]:
# There are several data points on the same day.
# We assume that data is uniformly distributed across the day.
# We will add a small time increment to each data point to make them unique.

def add_time_stamps(df):
    # Resample dates with duplicate indices by adding hours and minutes
    # Find duplicate dates in index
    duplicate_dates = df.index[df.index.duplicated(keep=False)]
    
    for date in set(duplicate_dates):
        duplicates = df.loc[date]
        # Calculate time increment based on the number of duplicates
        num_duplicates = len(duplicates)
        time_increment = datetime.timedelta(days=1) / num_duplicates

        # Assign new unique timestamps to each duplicate
        times = [date + i * time_increment for i in range(len(duplicates))]
        df.loc[date, 'new_index'] = times
    
    # Set the new unique datetime index
    df.set_index('new_index', inplace=True)
    df.index.name = 'datetime'  # Rename the index for clarity
    
    return df

In [3]:
def convert_to_u_v(speed, direction):
    # Convert direction from CW (North = 0°) to CCW (East = 0°)
    direction_ccw_deg = (90 - direction) % 360
    direction_rad = np.deg2rad(direction_ccw_deg)

    # Calculate u- and v-components
    u = speed * np.cos(direction_rad)
    v = speed * np.sin(direction_rad)

    return u,v

In [4]:
def load_and_save_CMEMS_data(obs_dir,platform_name):

    all_cur_list = []
    longitudes = []
    latitudes = []
    for o in obs_dir:
        ds = nc.Dataset(o)
        formatted_time = []
        start = datetime.date(1950,1,1)
        for days in ds["TIME"][:].data:
            formatted_time.append(start + datetime.timedelta(days = days))

        if "EWCT" in ds.variables.keys():
            ew_cur = ds["EWCT"][:].data
            ew_cur[ds["EWCT"][:].mask] = np.nan # Set fill values to nan
            ew_cur_da = np.nanmean(ew_cur,axis=1) # depth averaged East-West current
            ew_cur_da[(ew_cur_da>500) | (ew_cur_da<-500)] = np.nan # Set invalid values to nan

            ns_cur = ds["NSCT"][:].data
            ns_cur[ds["NSCT"][:].mask] = np.nan # Set fill values to nan
            ns_cur_da = np.nanmean(ns_cur,axis=1) # depth averaged North-South current
            ns_cur_da[(ns_cur_da>500) | (ns_cur_da<-500)] = np.nan # Set invalid values to nan
        elif "HCSP" in ds.variables.keys():
            cur_s = ds["HCSP"][:].data
            cur_d = ds["HCDT"][:].data

            cur_s[ds["HCSP"][:].mask] = np.nan # Set fill values to nan
            cur_d[ds["HCDT"][:].mask] = np.nan # Set fill values to nan

            u,v = convert_to_u_v(cur_s, cur_d)
            
            ew_cur_da = np.nanmean(u,axis=1) # depth averaged East-West current
            ns_cur_da = np.nanmean(v,axis=1) # depth averaged North-South current
            # ew_cur_da[(ew_cur_da>200) | (ew_cur_da<-200)] = np.nan # Set invalid values to nan


        lon = np.nanmean(ds["LONGITUDE"][:].data)
        lat = np.nanmean(ds["LATITUDE"][:].data)
        longitudes.append(lon)
        latitudes.append(lat) 

        cur = pd.DataFrame({"u": ew_cur_da,"v": ns_cur_da}, index = pd.DatetimeIndex(formatted_time))
        try:
            quality_filter = ((ds["HCSP_QC"][:].data<=2).any(axis=1)) & ((ds["HCDT_QC"][:].data<=2).any(axis=1))
        except IndexError:
            quality_filter = ((ds["EWCT_QC"][:].data<=2).any(axis=1)) & ((ds["NSCT_QC"][:].data<=2).any(axis=1))
        cur = cur[quality_filter]
        cur = cur[(cur.index >"2021-12-31") & (cur.index <"2024-01-01")]

        #  Add timestamps
        cur = add_time_stamps(cur)
        
        all_cur_list.append(cur)

    all_cur=pd.concat(all_cur_list)
    # Replace missing entries with nan such that we have a full date time index
    freq=all_cur.index[1]-all_cur.index[0]
    all_cur = all_cur.set_index(all_cur.index).resample(freq).sum().replace(0.00, np.nan) 
    all_cur = np.round(all_cur.dropna(),decimals=4)
    all_cur.index.name = "datetime_UTC"
    all_cur.to_csv(f"../observations/{platform_name}_u_v.csv")

    
    if Path("../observations/current_stations.csv").exists():
        locations_all = pd.read_csv("../observations/current_stations.csv",index_col=0)
        locations_all.loc[platform_name] = [np.round(np.mean(longitudes),decimals=6),np.round(np.mean(latitudes),decimals=6)]
    else:
        locations_all = pd.DataFrame({"Longitude": [np.round(np.mean(longitudes),decimals=6)],
                              "Latitude": [np.round(np.mean(latitudes),decimals=6)]},index=[platform_name])
    
    locations_all.to_csv("../observations/current_stations.csv")

    return all_cur

List of remaining platforms:
- [] DB
- [] MO
- [] IJmondstroompaal2      GOOD

# DB

OBS: DB is outside of model domain

In [5]:
# The modelskill package can be used to compare model results with observations.
# For more info on modelskill, see https://github.com/DHI/modelskill
# obs_fldr = "raw_data/CMEMS/"
# # Collect observation directories in list
# obs_dir = [f"{obs_fldr}GL_TS_DB_6301615.nc"]

# ds = nc.Dataset(obs_dir[0])
# ds.variables.keys()

In [6]:
# load_and_save_CMEMS_data(obs_dir,"DB")

# MO

In [7]:
obs_fldr = "raw_data/CMEMS/"
# Collect observation directories in list
obs_dir = [f"{obs_fldr}NO_TS_MO_6201065.nc"]

ds = nc.Dataset(obs_dir[0])
ds.variables.keys()

dict_keys(['TIME', 'TIME_QC', 'DEPH', 'LATITUDE', 'LONGITUDE', 'STATION', 'OSAT', 'OSAT_QC', 'OSAT_DM', 'TEMP', 'TEMP_QC', 'TEMP_DM', 'HCSP', 'HCSP_QC', 'HCDT', 'HCDT_QC', 'PSAL', 'PSAL_QC', 'PSAL_DM'])

In [8]:
ds["HCSP"]

<class 'netCDF4.Variable'>
float32 HCSP(TIME, DEPTH)
    standard_name: sea_water_speed
    units: m s-1
    _FillValue: 9.96921e+36
    long_name: Horizontal current speed
    valid_min: 0.001
    valid_max: 1.5
    coordinates: TIME LATITUDE LONGITUDE DEPH STATION
    ancillary_variables: HCSP_QC
    data_mode: R
unlimited dimensions: 
current shape = (1193772, 30)
filling off

In [9]:
ds["HCDT"]

<class 'netCDF4.Variable'>
float32 HCDT(TIME, DEPTH)
    standard_name: direction_of_sea_water_velocity
    units: degree
    _FillValue: 9.96921e+36
    long_name: Current to direction relative true north
    valid_min: 0.0
    valid_max: 360.0
    coordinates: TIME LATITUDE LONGITUDE DEPH STATION
    ancillary_variables: HCDT_QC
    data_mode: R
unlimited dimensions: 
current shape = (1193772, 30)
filling off

In [10]:
ds["TIME"]

<class 'netCDF4.Variable'>
float64 TIME(TIME)
    long_name: Time
    standard_name: time
    units: days since 1950-01-01T00:00:00Z
    valid_min: -90000.0
    valid_max: 90000.0
    axis: T
    ancillary_variables: TIME_QC
    calendar: standard
unlimited dimensions: 
current shape = (1193772,)
filling off

In [11]:
load_and_save_CMEMS_data(obs_dir,"MO_NN")

C:\Users\frtp\AppData\Local\Temp\ipykernel_43080\627281904.py:32: RuntimeWarning: Mean of empty slice
  ew_cur_da = np.nanmean(u,axis=1) # depth averaged East-West current
C:\Users\frtp\AppData\Local\Temp\ipykernel_43080\627281904.py:33: RuntimeWarning: Mean of empty slice
  ns_cur_da = np.nanmean(v,axis=1) # depth averaged North-South current


,u,v
datetime_UTC,,
2022-01-01 00:00:00,-0.3121,0.0813
2022-01-01 00:12:00,-0.3254,0.1012
2022-01-01 00:24:00,-0.3145,0.0920
2022-01-01 00:36:00,-0.3498,0.0860
2022-01-01 00:48:00,-0.3303,0.0713
...,...,...
2023-12-31 23:00:00,0.6634,0.1102
2023-12-31 23:12:00,0.6220,0.1549
2023-12-31 23:24:00,0.5882,0.1134


# IJmondstroompaal2

Skip this station, since it is too close to shore

In [14]:
# obs_fldr = "raw_data/CMEMS/"
# # Collect observation directories in list
# obs_dir = [f"{obs_fldr}NO_TS_MO_IJmondstroompaal2.nc"]

# ds = nc.Dataset(obs_dir[0])
# ds.variables.keys()

In [15]:
# load_and_save_CMEMS_data(obs_dir,"IJmondstroompaal2")